In [1]:
genom = """CTAACCCTAA
CCCTAACCCTAACCCTAACCCTAACCCTCTGAAAGTGGACCTATCAGCAGGATGTGGGTGGGAGCAGATT
AGAGAATAAAAGCAGACTGCCTGAGCCAGCAGTGGCAACCCAATGGGGTCCCTTTCCATACTGTGGAAGC
TTCGTTCTTTCACTCTTTGCAATAAATCTTGCTATTGCTCACTCTTTGGGTCCACACTGCCTTTATGAGC
TGTGACACTCACCGCAAAGGTCTGCAGCTTCACTCCTGAGCCAGTGAGACCACAACCCCACCAGAAAGAA
GAAACTCAGAACACATCTGAACATCAGAAGAAACAAACTCCGGACGCGCCACCTTTAAGAACTGTAACAC
TCACCGCGAGGTTCCGCGTCTTCATTCTTGAAGTCAGTGAGACCAAGAACCCACCAATTCCAGACACACT
AGGACCCTGAGACAACCCCTAGAAGAGCACCTGGTTGATAACCCAGTTCCCATCTGGGATTTAGGGGACC
TGGACAGCCCGGAAAATGAGCTCCTCATCTCTAACCCAGTTCCCCTGTGGGGATTTAGGGGACCAGGGAC
AGCCCGTTGCATGAGCCCCTGGACTCTAACCCAGTTCCCTTCTGGAATTTAGGGGCCCTGGGACAGCCCT
GTACATGAGCTCCTGGTCTGTAACACAGTTCCCCTGTGGGGATTTAGGGACTTGGGCCTTCTGTCTTTGG
GATCTACTCTCTATGGGCCACACAGATATGTCTTCCAACTTCCCTACACAGGGGGGACTTCAAAGAGTGC
CTTGAGCTGATCTGGTGATTGCTTTTTTGTACTGTTATTTATCTTATTCTTTTCATTGTGAGGTACTGAT
GCAAACACTTTGTACGAAAAGGTCTTTCTCATCTCGGGAGTCCCCGTCTATTTGTCCCGGTCCCTGTTAA
CCCAGTCCCCGACAGGAGCCCCTTCTGCACCTTGAGCTCTCACCACTCACCGTCCATCCAGCCCCAGCTC
TGCCTGCAACCCACCCATCCCTGGGACTCGGGCCTCCCCTCTCTAGTGGTCTGGTCATCAGGCCAGGGGC
ACGTGGAAGAAGCTATCGTGGCAAAGGGAGCAGTCATATCCCCAAAATCTGTGGTTGGTTTACCACCACC
ATGGAAACCCCAGGGTGGGACTCTAGTTTCAGGTTGGAGCTGAGCCCTGTCGGGAATGAGCTTTCCCCAG
CTATGGCTTCTTGGGGCCCCTGTGCCCTGAGCTGTGTCTCCCAGCATCGGGTCCCCACCATGCATATGGC
CCACTCAGGCACAGTGCCGCGATGGCTGCATGCGTGAGGGGGGCCTGGGCCCAGGGCTGGGAGTCCTTTG
TGTCTCATGGCCATGATTGTCCTTCCGAGTATGATATGGTGGCCAATTTCTTTTATTCTGTCGTTCAGAG
TGAGTAAATGATGTAGAGTTCATGCAGAAAAAAATACAACAAAAACCAAGGGAACATAGAATTGGAAAAC
GCGTCACAGCAATGAGTTAAATAGGTAACAAATTTCATCATTTGAAGAAAGACTTAGAGTGCCAAAAGTG
CCTCTTAAGTCTCCTTTAAAAAGTAGCAAAATTCATCCCTGAAGAAGCATCTTGGCCTTTTTCATGTACT
CAGAGTGCTGGTGAAGAACAAAGATTGCTGAAACATTATGTACCTAACAGCGTTACAGGGTGTAGATAAC
ACACTGGAAAACCTGGTCGTTACAGTGGACATATTCCAGGAAGTCCTTGCCTGAGGTTTTCCAAGTTATG
GAATTGCTTGAGATTGGAAGAGGCGATGGAGGGTACAACTGTAATGCCCAACCTCATTTTTGCTAACCCT
GTTTTTAGACTCTCCCTTTCCTTCAATCACCTAGCCTTCTTTCCACCTGAAAGGACTCTCCCTTAACTGA
GAGAACCGGACAGACTCCATCTTGGCTCTTTCACTGGCAGCCCCTTCCTCAAAGACTTAACTCGTGCAAG
CTGACTCCCAGGACATCCGAGAATGCAATTAACTGACAACCTACTGTGGCGAGCTACATCCGCAGTCCCC
AGGAATTCGTCCGATTGATAACGCCCAATTACCCGCGTCTATCACCTTGTAATAGTCTTAAAGCACCTGC
ACCTGGAACTGTTTACTTTCCTGTAACCATTTATCCTTTTAACATTTTGCCTGATTTACTTATGTAAAAT
TCTTTTAACTAGACCGCCACTCCCCTTTCTAAACAAAAGTATAAAAGAAAATCTAGCCCCTTCTTTGGGA
CTGAGACAATTTTGAGGTTAACGCAGGGTGCCTGTAATCCTAAGGGAGGAGACCGCCACTTCTGCTGCCC
TTCCCTTCCCCACACCCCCTTCTCTAGTTTATGAAACAGGGAAAAAGGGAGAAAGCAAAAAGATAAAAAA
AACAGAAGTAAGATAAATAGCTAGACGACCTTGGCAGCACCACCCGGCACTGGTGGTTAAAATAATAATA
ATAATAATATTAACCCCTGACCTAAACTACTTGTGTTATCTGTAAATTCCAGACACTGTATGAGGAAGCC
CTGCAAAACTTTCTGTTCTGTTATCTGATGCGTGTAGCCCCCAGTCACGTTCCGATGCTTGCTCGATCTA
TCACGACCCTTTCAAGTGAACCCCTTAGAGTCGTAAACCCTTAAAAGGGCCAGGAATTTCGTTTTCGGGG
AGCTCGGCTCTTCAGGCCCAAGTAAACCTGCCGTATCTCACCTGAGACCAACCCCCAACTACAAAACTCA
ACCTGGAATTTTCCCAGGACCAAACCCATCTATATTCTGTAACCCGAAACCTCAAAGCCTAACCCTAACC
CTAACCCCTACAGTTGAGGTCCCCCCGCCCCTGTGGTTCCAGCTCAAGACAACCTGCCCCTCCGTGGGTT
TGCAGGCCCTCTGGTGGGGGTGGGAGCTGGGGGCCACATACAGCTCTCTGAGCTTAAGCCATTTTCTTCC
TTCATTCCTTCCTTCCTCCCTCCCTTCCCTTCCTTCCTCCCTCCCTCCCTCCTTCCCTCCCTCCCTTTTT
TTTTTCAGGGTCTTGCTCTGTCACCCAGGCTGGAGTGCAGTGGCATGATCACAGCTGACTGCAGCCTCGG
ACTCCCAGGCTCAAGTTATCTTCCCTCCTCAGCCTCCAGAGTAGGTGGGACTACAGGAGTGTGCCATCGC
ACCCAGCGAATTTCTTAATTTTTATTTTGTAGAGATGAGGCCTCTCGATATTGTCCAGGCTTGGAAGTAG
TTCTTGAAATTCAAGAGGATCTTGAAGTTCTGACCTCCTGTCAATATCCCTTCCCCTCACCTTGACCCTC
CCATTCTGCCCCACCTGTCAGGATCACAAGGACCCCCAGATCAGCAGATGGGAACCGGACCAAAAAGAGA
AATAGTGCTGTCCCGCCCATAAGTACCACCCCCAGACTCCCTGCTCCACCCTCTGGACCACAGGGAGGCC
CCATGCTCCATCCCTGAAAACCACCCCCAGACCCCCTGCTCCTCCCCACGGGACCGCCCCCAGATCCACT
GTTCCTGTCCTCCGGACCACACCGGACAGCTCCTTCCCTCGGCGCCATCCCCAGACCCCAGCTCCTCCCC
TCAGGATCATCCCCAGACCCCCGCTCCTCCCATCAGGACCGCCCCCAGAACCCCCTGCTCCTCCCCACAG
GACCACCCCCAGACCCCCGCTCCTCCCCTAAGGACCACCCACAGACCCCCACGCTTCCCCTCGGGACCAC
CCCCAGACCCCCGCTCCTCCCCTCAGGACCACCCCCAGATCCACTGTTCCTGTCCTCCGGACCACACCAG
ACAGCTCCTTCCCTCGGCGCCATCCCCAGACCCCCACGCTTCCCCTCGGGACCACCCCCAGACCCCCGCT
CCTGCCCTCGGGACCACGCCCAGACCCCCTGCTCCTCCCCACGAAACCACGCCCAGTAACCCCCCTCTTC
TCCCCTCAGGACCACCCCCAGACCCCCGCTCCTCCCCTGGGAACCACCCCCAGACCCCCTGCTCCTCCCC
ACGAAACCACCCCCAGACTCCCTGCTCCTGTCCTCCGGACCACGCGACTGCTCCTTCCCTCAGCGCCACC
CCCAGACTGCCGCTCCTCCTGTCGGGACCCCCTGAGGCTTTCTCCACCCGGAGTGCGGGGTAGGGAGCAG
ACGGAGAGTGACGGAGGGTGACGGAGAGTGACGAAAGTAGACGATGTCTGACGGAGAAGAGCCGAGCGGA
GCTGAAGGGCGGCGGAGAGTGACGGAAAGTGGCGAGAATTGACGGAAAGTGACGGGGACTGACGGACAGT
GACGAAGAGTCACGAAATTTATCAGAGGGCGACAAAGAGGAAAGCGAAATGGTGAGATGCAGCCGGCCGA
GCCTAATCGGAGATGACGGAAAGTGACGGAGAGGAACGAGGAGTAAAGAGGGGTGACGAAAAGAGCCGAA
GCTGGTGGAGGCGAAGAACTGAGTGAGGGAAGATAGCCGAGATTAGCGGGTGGGCTGCAGCCGGGGCAGT
CGCCCGAATGGGCGGGACCCCACGGAGTTAGCGAGAGGATGCGAACAGCGGCCAGCCGGGCAGCACGCGA
GCGAGGGAGGGCGTGGAGGGCCGTGGGTCCGCCTGCACTGAGGCAGGCATGCGTGGCACCGAGGTGACCC
GGGTGGGAGGTGCACCGCCGCCCCCTGGCAGTCTCTCCGCGGAGCCCAAGCCCGTCTTCTCCGCCCCTTT
GCAGACCTCGGCGCCCAGCCTGGCCCCTGACGCCCACCCGCGGCCCCACCCAGCGCCCGGGCCCACGAGG
CCGAGGAGCGGCGGAGACTAACGGCCCCTGGACCCCAGGCAGCACATGGCCCAGAGCATCCCAGCCCAGT
GGAGGGCGGCACATGGCGGGAGGGCGGGAGTCCGTGTCCACTCATGGCCGGGGAGGGGAGGGCAAGTTCT
GGTGGCTGGGGAGGCCTAGAGCATCACAGCCCAGTGGAGGGCTGCACATGGCAGGGGAGGGGAGAGCAAG
AGTGTGCGGGCGAGGGAGGAGAGGGCAAGACTGCGTGTCCGCTCCAGTCTCTCTTCCTCATCTTATAAAG
CCACGAGTCCCATGACTGGGGACCCAACCTAATAACATTATCTAATCCTAATTGCCTCCCAAAGGCCATA
TCTCCAATCAGGACATGAATTCGGGGATTAAATTGCCAACACATGGCTGGGCGCGATGGCTTGTGCTTTT
TTTTTTTTTTTTTTTGAGACGGAGTCTCGCTCTGTCGCCCAGGCTGGAGTGCAGTGGCGCTATCTCGGCT
CACTGCAAGCTCTGCCTCCCGGGTTCACGCCATTCTCCTGCCTCAGCCTCCTGAGTAGCTGGGACTACAG
GCGCCCGCCACCACGCCCAGGTAATTTTTTTTTTTTTTTTTTTGTATTTCTTTGTAGAGACGGGGTTTCA
CCATGTTAGCCAGGATGGTCTCGATCTCCTGACCTCGTGATCCACCCATCTCGGTCTCCCAAAGTGCTAG
GATTGCAGGCCTGAGCCACCGCACCCAGCTGCCTTGTGCTTTTAATCCCAGCACTTTCAGAGGCCAAGGC
AGGCGATCACCTGAGGTCAGGAGTTCAAGACCAGCCTGGCCAACATGGTGAAACCCCATCTCTAATACAA
ATACAAAAAAAAAACAAAAAACGTTAGCCAGGAATGAGGCCCGGTGCTTGTAATCCTAAGGAAGGAGACC
ACCACTCCTCCTGCTGCCCTTCCCTTCCCCACACCGCTTCCTTAGTTTATAAAACAGGGAAAAAGGGAGA
AAGCAAAAAGCTTAAAAAAAAAAAAAAAAAAACAGAAGTAAGATAAATAGCTAGATGATCTTGGCAGCAC
CACCCGACCCTGGTGGTTAAAATAATAATAATAATAATATTAACCCCTGACCTAAACTACTTGTGTTATC
TGTAAATTCCAGACACTGTATGAGGAAGCCCTGCAAAACTTTCTGTTCTGTTATCTGATGCGTGTAGCCC
CCAGTCACGTTCCCCATGCTTCCTCGATCTATCACAACCCTTTCACGTAAACCCCTTAGAGTTGTAAACC
CTTAAAAGGGCCAGGAATTTCGTTTTCGGGGAGCTTGGCTCTTCAGGCGCAAGTGTGCCGATGCTCCTGG
CAGAGTAAAGCCCTTCCTTCTTTAACCCAGTGTCTGAGGAATTTTGTCTGCGGCTTGTCCTGCCACAATG
CCAGCTACCAGGAGGCTGAGGTGTGAGAATCGCTCGAACCTGGGAGGCAGAGGTTGCAGCGAGCCGAGAT
CCCACGATTGCACTCCAGCCTGGGTGATAGATGACAGAGCAAGACACCATGTCAAAAAGAAAAGAAAAAA
AATTGCCAACACATGAAATCTGGAGAACACATTCAAGCAATAGCACCATATAAATCTGTACATACCTTGC
AAACACTGTAAACCTGCCGCATCTCACCTGAGACCAACCCCCAACCGCAAAACCCAGACCTGGAATTTTC
CCAGGACCAAACCCATCTATATTCTGTAACCCGAAATCTCAAAGCCTAACCCTAACCCTAACCCCGACAG
TTCAGGTCCCCCCGTCCCCGTAGCTCCAGCTCAAGACAACCTGCCCCTCCGTGGGTTTGCAGGCCCTTTG
GTGGGGGTGGGAGCTGGGGGCCACACACAGCTCTCTGAGCTTAAGCCATTTCCTTCCTTCCTTCATTCCT
TCCTTCCTCCCTCCCTCCCTCCTTCCCTCCCTCCCTTTTTTTTTTCAGGGTCTTGCTCTGTCACCCAGGC
TGGAGTGCAGTGGCATGATCACAGCTGACTGCAGCCTCGGACTCCCAAGCTCAAGCGATCTTCCCGCCTC
AGCCTCCAGAGTAGGTGGGACTACAGGAGTGTGCCATCGCACGTGGCGAATTTCTTAATTTTTATTTTGT
AGAGATGAGGCCTCCCGATATTGTCCAGGCTTGGAAGTAGTTCTTGAAATTCAAGAGGATCTGGAAGTTC
TGGTCTCCTGTCAATATCCCTTTCTAAGATGAAATGTAGAATCCTCTAATCACCCCAGGATACCTTTTCC
AGCTATATTACCCTTTCCTTTCAGCCGACTAGAGTGGAAGCTCAGCAAAGAACGGATCTATGCCTGCACA
CAGGCAGCTTTGAACTCACTGAGATCTATAGAAGAATTTTTTAAAATTTGCCAGAATGAAAAAGAAGTAT
CCTCCAGTGAGCCTCAAGGACAGCCAGGGAAGGCAGCTACTGGGACCTGCCACTGGCCAGGATTGAGAGA
GTCTGAGGTGTCCATTCTCCTATGGATGGGTCACACATCAGAACTGAACCACAAACCCTTTCCTGCTCCC
TTGGGACCAGGCTGCCACTCCTTCCTTCCCCTCACCTTGACCCTCCCATTCGGCCCCACCTGTCAGGATC
ACAAGGACCCCCAGATCAGCAGATGGGAACCGGACCAAAAAGAGAAATAGCGCTGTCCCGCCCAAAAGTA
CCAACCCCAGACCCCCTGCTCCACCCTCCGAACCACAGGGAGACCCCATGCTCCACCCCTGAAAACCACA
TGCAGACCCCCGCTCCTCCTCTTAGGGCCATCCCCAGACCCCCCCTCCTACCCACGGGACCACACCCAGA
ACCCCCAGCTCCTCCCCTCGGGACCACCCCAGACCCCCGCTCCTCCCCTCGGGACCACCCCCAGACCCCC
GCTCCTCCCCTCGGGACCACCCCAGACCCCCGCTCCTCCCCTCGGGACCACCCCAGACCCCCTGCTCCTC
CTCTTAGGACCATCCCCAGACCCCCCCTCCTACCCACGGGACCACACCCAGAACCCCCAGCTCCTCCCCT
CGGGACCACCCCAGACCCCCGCTCCTCCCCTCGGGACCACCCCAGACCCCCGCTCCTCCCCTCGGGACCA
CCCCAGACCCCCGCTCCTCCCCTCGGGACCACCCCCAGACCCCCTGCTCCTCCTCTTAGGACCATCCCCA
GACCCCCCCTCCTACCCACGGGACCACACCCAGAACCCCCAGCTCCTCCCCTCGGGACCACCCCCAGACC
CCCTGCTCCTCCTCTTAGGACCACCCCAGACCCCCGCTCCTCCCCTCGGGACCACCCCAGACCCCCGCTC
CTCCCCTCGGGACCACCCCAGACCCCCGCTCCTCCCCTCGGGACCACCCCCGACCCCCTGCTCCTCCTCT
TAGGACCATCCCCAGACCCCCCCTCCTACCCACGGGACCACACCCAGAACCCCCAGCTCCTCCCCTCGGG
ACCACCCCAGACCCCCGCTCCTCCCCTCGGGACCACCCCAGACCCCCGCTCCTCCCCTCGGGACCACCCC
AGACCCCCGCTCCTCCCCTCGGGACCACCCCCAGACCCCCTGCTCCTCCTCTTAGGACCATCCCCAGACC
CCCCCTCCTACCCACGGGACCACACCCAGAACCCCCAGCTCCTCCCCTCGGGACCACCCCAGACCCCCGC
TCCTCCCCTCGGGACCACCCCAGACCCCCGCTCCTCCCCTCGGGACCACCCCAGACCCCCGCTCCTCCCC
TCGGGACCACCCCCAGAACCCCCGCTCCTCCCCTCGGGACCACCCCAGACCCCCGCTCCTCCCCTCGGGA
CCACCCCAGACCCCCGCTCCTCCCCTCGGGACCACCCCCAGACCCCCTGCTCCTCCTCTTAGGACCATCC
CCAGACCCCCCCTCCTACCCACGGGACCACCCCCAGACCCCCCGCTCCTCCCCTCGGGACCACCTCAGAC
CCCCGCTCCTCCCCTCGGGACCACCCCAGACCCCCGCTCCTCCCCTCGGGACCACCCCAGACCCCCTGCT
CCCATTTTCCAGACCACACCAGACGGCTTCTTCCCTCGGCACCACCCCCAGACTGCCGCTCCTCCTATCG
GGACCTCCTCGGGGCCATCTCCAGACCCTCCCCCCGCTCCACCCCTCGGGTCCCTCTCGGGACCACCCTC
""".replace('\n', '')
"""
...
AGGTCTCATTGAGGACAGATA""".replace("\n", "")
print()

In [2]:
read_length = 200
coverage = 5
gen_len = len(genom)

In [3]:
all_reads = gen_len - 1 - read_length

one_read_prob = read_length / gen_len

reads_num = 1

while one_read_prob * reads_num < coverage:
    reads_num += 1

reads_num

223

In [4]:
import random

reads = []
for _ in range(reads_num):
    start = random.randint(0, gen_len - read_length)
    reads.append(genom[start:start+read_length])

In [5]:
def join(first : str, second :str) -> str:
    joined = None
    for length in range(1, max(len(first), len(second))):
        try:
            if first.endswith(second[:length]):
                joined = first + second[length:]
            elif second.endswith(first[:length]):
                joined = second + first[length:]
        except:
            pass

    return joined

In [6]:
import itertools

class Cache:
    def __init__(self, strings):
        self._cache = dict()

        for first, second in itertools.product(strings, repeat=2):
            if first == second:
                continue
        
            joined = join(first, second)
            
            if len(frozenset([first, second])) == 1:
                print(frozenset([first, second]))

            if joined:
                self._cache[frozenset([first, second])] = joined

    def remove(self, string):
        for key in list(self._cache):
            if string in key:
                del self._cache[key]

    def add(self, string, strings):
        for other in strings:
            if string == other:
                continue

            joined = join(string, other)

            if not joined:
                continue
            
            if string == joined or other == joined:
                continue
            
            self._cache[frozenset([string, other])] = joined

    def get(self, first, second) -> str:
        key = frozenset([first, second])

        if key in self._cache:
            return self._cache[key]
        else:
            return None

    def get_max(self):
        if self._cache:
            return max(self._cache.keys(), key=lambda x: len(self._cache[x]))
        else:
            return None, None

In [7]:
reads_set = set(reads)

In [8]:
cache = Cache(reads_set)

In [9]:
import itertools
from collections import namedtuple
Contig = namedtuple('Contig', ['first', 'second', 'contig'])

while len(reads_set) != 1:

    maxContig = None

    before = len(reads_set)

    key = list(cache.get_max())
    first, second = key[0], key[1]
    joined = cache.get(first, second)

    if not joined:
        continue

    if not maxContig:
        maxContig = Contig(first, second, joined)
    elif joined != first and joined != second:
        maxContig = max(maxContig, Contig(first, second, joined), key=lambda x: len(x.contig))

    # for first, second in itertools.product(reads_set, repeat=2):
    #     if first == second:
    #         continue
        
    #     joined = cache.get(first, second)

    #     if not joined:
    #         continue

    #     if not maxContig:
    #         maxContig = Contig(first, second, joined)
    #         continue

    #     if joined != first and joined != second:
    #         maxContig = max(maxContig, Contig(first, second, joined), key=lambda x: len(x.contig))

    if maxContig:
        reads_set.discard(maxContig.first)
        reads_set.discard(maxContig.second)
        reads_set.add(maxContig.contig)

        cache.remove(maxContig.first)
        cache.remove(maxContig.second)
        cache.add(maxContig.contig, reads_set)

        after = len(reads_set)
        print('len', before, '/', after)
    else:
        break

len 220 / 219
len 219 / 218
len 218 / 217
len 217 / 216
len 216 / 215
len 215 / 214
len 214 / 213
len 213 / 212
len 212 / 211
len 211 / 210
len 210 / 209
len 209 / 208
len 208 / 207
len 207 / 206
len 206 / 205
len 205 / 204
len 204 / 203
len 203 / 202
len 202 / 201
len 201 / 200
len 200 / 199
len 199 / 198
len 198 / 197
len 197 / 196
len 196 / 195
len 195 / 194
len 194 / 193
len 193 / 192
len 192 / 191
len 191 / 190
len 190 / 189
len 189 / 188
len 188 / 187
len 187 / 186
len 186 / 185
len 185 / 184
len 184 / 183
len 183 / 182
len 182 / 181
len 181 / 180
len 180 / 179
len 179 / 178
len 178 / 177
len 177 / 176
len 176 / 175
len 175 / 174
len 174 / 173
len 173 / 172
len 172 / 171
len 171 / 170
len 170 / 169
len 169 / 168
len 168 / 167
len 167 / 166
len 166 / 165
len 165 / 164
len 164 / 163
len 163 / 162
len 162 / 161
len 161 / 160
len 160 / 159
len 159 / 158
len 158 / 157
len 157 / 156
len 156 / 155
len 155 / 154
len 154 / 153
len 153 / 152
len 152 / 151
len 151 / 150
len 150 / 149
len 14

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
reads_set

{'ATAAAAGCAGACTGCCTGAG',
 'TAAATCTTGCTATTGCTCACTGCAGCTTCACTCCTGAGCAGAAAGAAGAAACTCAGAAGCCAGTGAGACCACAACCCCACCAG',
 'TAACCCTCTGAAAGTGGACCTTTAAGAACTGTAACAC',
 'TCCACACTGCCTTTATGAGC',
 'TCCGCGTCTTCATTCTTGAAGCCAGCAGTGGCAACCCAATACTGTGGAAGCTTCGTTCTTGAAGTCAGTGAGACCAGCAGTGGCAACCCAATGGACCTATCAGCAGGATGTGCCTGAGCCAGCAGTGGCAACCCCACCAGAAAGAAGAACTCCGGACGCGCCACCTTTCACTCTTTGGGTCCACACTGGGAGCAGATTAGAGAATAAGCTTCGTTCTTTCACTCTGCAATAAATCTTGCTATTGAAACTCAGAACACATCTGAACCCTAACCCTAACCCTAAGCAGATTAGAGAATAAAAGTGAGACCACAACCCCACCAGTGGACCTATCAGCAGGATAACCCTAACCCTAACCCTAAATCTTGCTATTGCTCACTGAGACCAAGAACCCACCAATGTGGGTGGGAGCAGATTAGGATGTGGGTGGGAGCAGACCCTCTGAAAGTGGACCTAGATTAGAGAATAAAAGCAGAGCCAGCAGTGGCAACCCAAAGCAGACTGCCTGAGCCATCTGAACATCAGAAGAAACCTAACCCTAACCCTCTGAAGTGGCAACCCAATGGGGTCTCCGGACGCGCCACCTTTAAGAAACAAACTCCGGACGCACTCACCGCGAGGTTCCGCACATCTGAACATCAGAAGAAAGGTCTGCAGCTTCACTCCAATGGGGTCCCTTTCCATCTGCAGCTTCACTCCTGAGAAGAAACTCAGAACACATCAGTGAGACCAAGAACCCACCCTAACCCTAACCCTCTGAGAGAATAAAAGCAGACTGCCACCAGAAAGAAGAAACTCTTTCACTCTTTGCAATAAATCTT

True